In [2]:
#!/usr/bin/env python
# coding: utf-8

import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import random
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()


# In[2]:


import geemap
import os
from google.cloud import storage
from google.cloud import client


# In[3]:


os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)


# Import assetts of interest

# In[4]:


geometry = ee.FeatureCollection('users/spotter/fire_cnn/raw/eurasia') #area of interest
mod1 = ee.ImageCollection("MODIS/061/MOD10A1") #active fire
mod2 = ee.ImageCollection("MODIS/061/MYD10A1") #active fire
fire_cci = ee.ImageCollection("ESA/CCI/FireCCI/5_1") #active fire
mod_burn = ee.ImageCollection("MODIS/061/MCD64A1") #mcd64a1
snow = ee.ImageCollection('MODIS/006/MOD10A1') #modis snow cover
# Load the MODIS water mask image and invert it.
water_mask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').Not()


sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") #sentinel 2
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
# s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/nbac_1985") #nbac_fire_polygons, this can be any polygon shapefile, final version would be nbac and mtbs
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/ann_w_id") #anna polygons 
# Cloud masking Sentinel-2
MAX_CLOUD_PROBABILITY = 50

def sent_maskcloud(image):
    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], 
                         ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).toShort()
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return image.updateMask(isNotCloud).reproject(crs=image.projection().crs(), scale=30)

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    primary=sent_2A,
    secondary=s2Clouds,
    condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
)
sent_2A = ee.ImageCollection(s2SrWithCloudMask).map(sent_maskcloud)

# Snow quality masking
def mask_quality_snow(image):
    qa = image.select('NDSI_Snow_Cover_Basic_QA')
    quality_mask = qa.bitwiseAnd(3).eq(0)
    return image.updateMask(quality_mask)

snow = snow.map(mask_quality_snow).select('NDSI_Snow_Cover')

# Add date bands
def add_date_bands(img):
    date = img.date()
    cal_doy = date.getRelative('day', 'year')
    rel_doy = date.difference(ee.Date.fromYMD(date.get('year'), 1, 1), 'day')
    millis = date.millis()
    date_bands = ee.Image.constant([cal_doy, rel_doy, millis]).rename(['calDoy', 'relDoy', 'millis'])
    return img.addBands(date_bands).cast({'calDoy': 'int', 'relDoy': 'int', 'millis': 'long'})

# Process year for last day of snow in spring
def process_year(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 7, 1)
    
    year_col = snow.filterDate(start_date, end_date)
    
    no_snow_img = year_col.map(add_date_bands).sort('millis').reduce(ee.Reducer.min()).rename(['snowCover', 'calDoy', 'relDoy', 'millis']).set('year', year)
    
    return no_snow_img.updateMask(no_snow_img.select('snowCover').eq(0)).select('calDoy').rename('last_spring_snow_day')

# Process year for first day of snow in fall
def process_year_fall(year):
    start_date = ee.Date.fromYMD(year, 8, 31)
    end_date = ee.Date.fromYMD(year, 12, 31)
    
    year_col = snow.filterDate(start_date, end_date)
    
    no_snow_img = year_col.map(add_date_bands).sort('millis', False).reduce(ee.Reducer.min()).rename(['snowCover', 'calDoy', 'relDoy', 'millis']).set('year', year)
    
    return no_snow_img.updateMask(no_snow_img.select('snowCover').eq(0)).select('calDoy').rename('first_fall_snow_day')

# Function to return the pre_fire and post_fire landsat and sentinel 2 data
def get_pre_post(pre_start, pre_end, post_start, post_end, geometry):
    lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    sent = sent_2A.filterDate(pre_start, post_end).filterBounds(geometry)

    def mask(image):
        qa = image.select('QA_PIXEL')
        mask = qa.bitwiseAnd(8).eq(0).And(qa.bitwiseAnd(10).eq(0)).And(qa.bitwiseAnd(32).eq(0))
        return image.updateMask(mask)

    def land_scale(image):
        return image.multiply(0.0000275).add(-0.2)

    def sent_scale(image):
        return image.multiply(0.0001)

    def to_float(image):
        return image.toFloat()

    # Process each satellite dataset
    def process_satellite(satellite, pre_start, pre_end, post_start, post_end, bands):
        pre_images = satellite.filterDate(pre_start, pre_end).map(mask).map(land_scale if satellite != sent else sent_scale).select(bands).map(to_float)
        post_images = satellite.filterDate(post_start, post_end).map(mask).map(land_scale if satellite != sent else sent_scale).select(bands).map(to_float)
        return pre_images, post_images

    pre_lt5, post_lt5 = process_satellite(lt5, pre_start, pre_end, post_start, post_end, ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])
    pre_le7, post_le7 = process_satellite(le7, pre_start, pre_end, post_start, post_end, ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])
    pre_lc8, post_lc8 = process_satellite(lc8, pre_start, pre_end, post_start, post_end, ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
    pre_sent, post_sent = process_satellite(sent, pre_start, pre_end, post_start, post_end, ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])

    return pre_lt5.merge(pre_le7).merge(pre_lc8).merge(pre_sent), post_lt5.merge(post_le7).merge(post_lc8).merge(post_sent)


# Main processing loop
all_ids = ee.List(lfdb.distinct(["ID"]).aggregate_array("ID")).getInfo()

all_ids = [52, 3266, 16086, 3381, 15894, 2464, 8737, 12494, 9763]
folder_name = 'anna_ndsi_pixel_wise'

for i in all_ids:
    fname = f"{folder_name}/median_{i}"
    
    stats = storage.Blob(bucket=bucket, name=fname).exists(storage_client)
    
    if not stats:
        sub_shape = lfdb.filter(ee.Filter.eq("ID", i))
        bbox = sub_shape.geometry().bounds()
        proj = ee.Projection("EPSG:4326")
        final_buffer = ee.Geometry.Polygon(bbox.coordinates(), proj).transform(proj)
        final_buffer2 = final_buffer.buffer(distance=5000).bounds()

        this_year = ee.Number(sub_shape.aggregate_array('Year').get(0)).getInfo()
        start_year = this_year - 1
        end_year = this_year + 1

        # Apply process_year and process_year_fall for the relevant years
        start_year_snow = process_year(start_year).clip(final_buffer)
        end_year_snow = process_year(end_year).clip(final_buffer)
        start_year_snow_fall = process_year_fall(start_year).clip(final_buffer)
        end_year_snow_fall = process_year_fall(end_year).clip(final_buffer)

        # Compute the last spring snow day and first fall snow day for each pixel
        last_spring_snow_day = ee.ImageCollection([start_year_snow, end_year_snow]).max()
        first_fall_snow_day = ee.ImageCollection([start_year_snow_fall, end_year_snow_fall]).min()

        # Shift the days to define the valid composite period per pixel
        start_day = last_spring_snow_day.add(7)
        end_day = first_fall_snow_day.subtract(7)

        def filter_by_snow_days(img):
            cal_doy = img.select('calDoy')
            mask = cal_doy.gte(start_day).And(cal_doy.lte(end_day))
            return img.updateMask(mask)

        # Apply the function to get the pre_fire and post_fire images
        pre_start = ee.Date.fromYMD(start_year, 6, 1)
        pre_end = ee.Date.fromYMD(start_year, 8, 31)
        post_start = pre_start.advance(2, 'year')
        post_end = pre_end.advance(2, 'year')

        # Apply the function to get the pre_fire and post_fire images
        all_imagery = get_pre_post(pre_start, pre_end, post_start, post_end, final_buffer)

        pre_input = ee.ImageCollection(all_imagery[0]).map(filter_by_snow_days).median()
        post_input = ee.ImageCollection(all_imagery[1]).map(filter_by_snow_days).median()

        raw_bands = pre_input.subtract(post_input).multiply(1000).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'], ['NBR', 'NDVI', 'NDII'])

        raw_bands = raw_bands.clip(final_buffer)

        # Export image to Cloud Storage
        task = ee.batch.Export.image.toCloudStorage(
            image=raw_bands.toShort(),
            region=final_buffer2, 
            description='median_' + str(i),
            scale=30,
            crs='EPSG:3413',
            maxPixels=1e13,
            bucket='smp-scratch',
            fileNamePrefix=fname
        )
        task.start()
        print(f"Downloading {fname}")


In [5]:
fname

'anna_ndsi_pixel_wise/median_52'

In [1]:
't'

't'

In [3]:
len(all_days_vi)

2